In [2]:
import math
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

WATER_COLOR = "#5A7B9A"
GROUND_COLOR = "#8B7964"
FONT = {"family": "sans-serif", "size": 14, "color": "black"}

def image_path(filename: str) -> Path:
    cwd = Path.cwd()
    root = cwd.parent if cwd.name == "resources" else cwd
    target = root / "images"
    target.mkdir(exist_ok=True)
    return target / filename

def add_horizontal_dimension(ax, x0, x1, y, label):
    """Adds a horizontal dimension line with arrows and a label."""
    ax.annotate("", xy=(x0, y), xytext=(x1, y),
                arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
    ax.text((x0 + x1) / 2, y + 0.15, label, ha="center", va="bottom", **FONT)

def add_bottom_dimension(ax, x0, x1, y, label):
    """Adds a horizontal dimension line below the object."""
    ax.annotate("", xy=(x0, y), xytext=(x1, y),
                arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
    ax.text((x0 + x1) / 2, y - 0.2, label, ha="center", va="top", **FONT)

def add_vertical_dimension(ax, x, y0, y1, label):
    """Adds a vertical dimension line."""
    ax.annotate("", xy=(x, y0), xytext=(x, y1),
                arrowprops=dict(arrowstyle="<->", color="black", lw=1.5))
    ax.text(x - 0.2, (y0 + y1) / 2, label, ha="right", va="center", **FONT)

def add_water_marker(ax, x, y, size=0.4):
    """Adds the inverted triangle water surface marker."""
    tm_1 = (x - size / 2, y + size)
    tm_2 = (x + size / 2, y + size)
    tm_3 = (x, y)
    marker = patches.Polygon([tm_1, tm_2, tm_3], closed=True,
                             facecolor="#3E5871", edgecolor="none")
    ax.add_patch(marker)

def add_slope_triangle(ax, anchor_point, size, z_label, invert=False):
    """
    Adds a slope indicator triangle.
    anchor_point: The corner of the channel wall (top point of the triangle).
    invert: False for left bank (slope down-right), True for right bank (slope down-left).
    """
    x, y = anchor_point
    
    if invert: # Left side slope
        p1 = (x, y)
        p2 = (x, y - size)
        p3 = (x - size, y - size) # z is horizontal run
        
        # Draw triangle
        triangle = patches.Polygon([p1, p2, p3], closed=True,
                                   facecolor=GROUND_COLOR, edgecolor="none")
        ax.add_patch(triangle)
        
        # Labels
        ax.text((p2[0] + p3[0]) / 2, p2[1] - 0.1, z_label, ha="center", va="top", **FONT)
        ax.text(p2[0] + 0.05, (p1[1] + p2[1]) / 2, "1", ha="left", va="center", **FONT)
        
    else: # Right side slope
        p1 = (x, y)
        p2 = (x, y - size)
        p3 = (x + size, y - size)
        
        triangle = patches.Polygon([p1, p2, p3], closed=True,
                                   facecolor=GROUND_COLOR, edgecolor="none")
        ax.add_patch(triangle)
        
        ax.text((p2[0] + p3[0]) / 2, p2[1] - 0.1, z_label, ha="center", va="top", **FONT)
        ax.text(p2[0] - 0.05, (p1[1] + p2[1]) / 2, "1", ha="right", va="center", **FONT)

def draw_rectangular_profile(filename="channel_profile_rectangular.webp"):
    b = 4.0
    y = 2.5
    ext = 1.0

    bl = (-b / 2, 0)
    br = (b / 2, 0)
    tl = (bl[0], y)
    tr = (br[0], y)
    
    # Extension lines for drawing
    ext_l = (tl[0] - ext, y)
    ext_r = (tr[0] + ext, y)

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.set_aspect("equal")
    ax.axis("off")

    # Water
    water_coords = [bl, br, tr, tl]
    ax.add_patch(patches.Polygon(water_coords, closed=True,
                                 facecolor=WATER_COLOR, edgecolor="none"))

    # Walls
    wall_x = [ext_l[0], tl[0], bl[0], br[0], tr[0], ext_r[0]]
    wall_y = [ext_l[1], tl[1], bl[1], br[1], tr[1], ext_r[1]]
    ax.plot(wall_x, wall_y, color="black", linewidth=2.5)

    # Annotations
    add_bottom_dimension(ax, bl[0], br[0], -0.5, "b")  # Bottom width
    add_horizontal_dimension(ax, tl[0], tr[0], y + 0.5, "T") # Top Width
    add_vertical_dimension(ax, br[0] + 0.5, 0, y, "y") # Depth
    add_water_marker(ax, 0, y, size=0.4)

    ax.set_xlim(ext_l[0] - 0.5, ext_r[0] + 0.5)
    ax.set_ylim(-1.0, y + 1.5)

    plt.savefig(image_path(filename), format="webp", dpi=300, bbox_inches="tight")
    plt.close(fig)

def draw_trapezoidal_profile(filename="channel_profile_trapezoidal.webp"):
    b = 3.0
    y = 2.5
    z_val = 1.0 # slope factor for drawing
    ext = 1.5

    bl = (-b / 2, 0)
    br = (b / 2, 0)
    tl = (bl[0] - z_val * y, y)
    tr = (br[0] + z_val * y, y)
    ext_l = (tl[0] - ext, y)
    ext_r = (tr[0] + ext, y)

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_aspect("equal")
    ax.axis("off")

    # Water
    water_coords = [bl, br, tr, tl]
    ax.add_patch(patches.Polygon(water_coords, closed=True,
                                 facecolor=WATER_COLOR, edgecolor="none"))

    # Walls
    wall_x = [ext_l[0], tl[0], bl[0], br[0], tr[0], ext_r[0]]
    wall_y = [ext_l[1], tl[1], bl[1], br[1], tr[1], ext_r[1]]
    ax.plot(wall_x, wall_y, color="black", linewidth=2.5)

    # Annotations
    add_bottom_dimension(ax, bl[0], br[0], -0.5, "b") # Bottom width
    add_horizontal_dimension(ax, tl[0], tr[0], y + 0.6, "T") # Top width
    add_vertical_dimension(ax, tr[0] + 0.5, 0, y, "y")
    
    # Slopes (z1 left, z2 right)
    add_slope_triangle(ax, tl, 0.7, "z1", invert=False)
    add_slope_triangle(ax, tr, 0.7, "z2", invert=True)
    
    add_water_marker(ax, 0, y, size=0.4)

    ax.set_xlim(ext_l[0] - 0.5, ext_r[0] + 0.5)
    ax.set_ylim(-1.0, y + 1.5)

    plt.savefig(image_path(filename), format="webp", dpi=300, bbox_inches="tight")
    plt.close(fig)

def draw_triangular_profile(filename="channel_profile_triangular.webp"):
    y = 3.0
    z_val = 1.2
    ext = 1.5

    apex = (0, 0)
    apex2 = (0, -0.4)
    tl = (-z_val * y, y)
    tr = (z_val * y, y)
    ext_l = (tl[0] - ext, y)
    ext_r = (tr[0] + ext, y)

    fig, ax = plt.subplots(figsize=(9, 5))
    ax.set_aspect("equal")
    ax.axis("off")

    # Water
    water_coords = [apex2, tr, tl]
    ax.add_patch(patches.Polygon(water_coords, closed=True,
                                 facecolor=WATER_COLOR, edgecolor="none"))

    # Walls
    wall_x = [ext_l[0], tl[0], apex[0], tr[0], ext_r[0]]
    wall_y = [ext_l[1], tl[1], apex[1]-0.4, tr[1], ext_r[1]]
    ax.plot(wall_x, wall_y, color="black", linewidth=2.5)

    # Annotations
    add_horizontal_dimension(ax, tl[0], tr[0], y + 0.6, "T")
    add_vertical_dimension(ax, tr[0] + 0.6, 0, y, "y")
    
    # Slopes (z1 left, z2 right)
    add_slope_triangle(ax, tl, 0.8, "z1", invert=False)
    add_slope_triangle(ax, tr, 0.8, "z2", invert=True)
    
    add_water_marker(ax, 0, y, size=0.4)

    ax.set_xlim(ext_l[0] - 0.6, ext_r[0] + 0.6)
    ax.set_ylim(-0.5, y + 1.5)

    plt.savefig(image_path(filename), format="webp", dpi=300, bbox_inches="tight")
    plt.close(fig)

def draw_circular_profile(filename="channel_profile_circular.webp"):
    D = 4.0
    R = D / 2
    # To match the reference image geometry where the triangle is clearly 
    # visible between center and water surface, we choose a depth < Radius
    y = 1.5 
    
    fig, ax = plt.subplots(figsize=(6, 7))
    ax.set_aspect("equal")
    ax.axis("off")

    # Draw Circle Outline
    # Shift circle so bottom is at y=0 to match hydraulic conventions (y=depth)
    center = (0, R) 
    circle = patches.Circle(center, R, fill=False, edgecolor="black", linewidth=2.5)
    
    # Calculate Water Geometry
    # Equation of circle: x^2 + (y-R)^2 = R^2
    # At water level y: x = +/- sqrt(R^2 - (y-R)^2)
    dy = y - R # distance from center to water level (negative if below center)
    half_chord = math.sqrt(R**2 - dy**2)
    
    # Draw Water
    # We use numpy to generate points along the arc from angle to angle
    # Angle calculation
    theta_rad = math.acos(-dy/R) # Angle from vertical bottom to water edge
    # Full circle angles 
    # We need the arc below the chord. 
    # Start angle is on the right (3pi/2 - theta?), End is on left.
    # Let's use simple trigonometry relative to center.
    # Right intersection angle relative to center (0 is right, pi/2 is top)
    # sin(alpha) = dy/R. 
    alpha = math.asin(dy/R) 
    start_angle = alpha # right side (radians)
    end_angle = -math.pi - alpha # left side (radians)
    
    angles = np.linspace(start_angle, end_angle, 100) # Draw bottom arc
    # However, fill logic is easier with polygon
    x_arc = R * np.cos(angles)
    y_arc = R + R * np.sin(angles) # Shifted by center Y
    water_coords = np.column_stack((x_arc, y_arc))
    ax.add_patch(patches.Polygon(water_coords, closed=True,
                                 facecolor=WATER_COLOR, edgecolor="none"))

    ax.add_patch(circle)

    # --- Annotations matching the reference image ---
    
    # 1. Dashed Centerline
    ax.plot([0, 0], [0, D], color="black", linestyle="-.", linewidth=1)
    
    # 2. Radius Lines to water surface
    ax.plot([0, half_chord], [R, y], color="black", linewidth=1) # Right
    ax.plot([0, -half_chord], [R, y], color="black", linewidth=1) # Left
    
    # 3. Label Radius (r)
    # Place label on the right radius line
    ax.text(half_chord/2 + 0.1, (R+y)/2, "r", ha="left", va="bottom", **FONT)

    # 4. Theta Angle (Arc at center)
    # The angle theta in the diagram is the full angle at the center
    # Create an arc patch
    # Angle from vertical down (270 deg)
    # We need the wedge between the two radii
    deg_offset = math.degrees(math.atan2(half_chord, abs(dy)))
    arc = patches.Arc(center, 1, 1, angle=-90, theta1=-deg_offset, theta2=deg_offset, 
                      color="black", linewidth=1.2)
    ax.add_patch(arc)
    ax.text(0.15, R - 0.15, r"$\theta$", ha="center", va="top", **FONT)

    # 5. Label z (vertical distance from center to water chord)
    # Draw a dimension line or just text. Image shows a vertical line segment labeled z.
    # ax.annotate("", xy=(0, y), xytext=(0, R),
                # arrowprops=dict(arrowstyle="-", color="black", lw=1))
    # ax.text(-0.2, (y+R)/2, "z", ha="right", va="center", **FONT)

    # 6. Dimensions D, y, T
    add_vertical_dimension(ax, R + 0.8, 0, D, "D") # Total Diameter
    add_vertical_dimension(ax, R + 0.5, 0, y, "y") # Water Depth
    add_horizontal_dimension(ax, -half_chord, half_chord,  2*R+0.3, "T") # Top Width

    # Water marker
    add_water_marker(ax, -R*0.9, y, size=0.35)

    pad = 1.0
    ax.set_xlim(-R - pad, R + pad)
    ax.set_ylim(-0.5, D + 0.5)

    plt.savefig(image_path(filename), format="webp", dpi=300, bbox_inches="tight")
    plt.close(fig)

def generate_all_profiles():
    draw_trapezoidal_profile()
    draw_triangular_profile()
    draw_rectangular_profile()
    draw_circular_profile()

if __name__ == "__main__":
    generate_all_profiles()